In [1]:
# -*- coding = utf-8 -*-
"""
Main function to build recommendation systems.
"""
import utils
from ItemCF import ItemBasedCF
from LFM import LFM
from BasicSVD import BasicSVD
from UserCF import UserBasedCF
from dataset import DataSet
from most_popular import MostPopular
from random_pred import RandomPredict
from utils import LogTime


def run_model(model_name, dataset_name, test_size=0.3, clean=False, user_list = [1, 100, 233, 666, 888]):
    print('*' * 70)
    print('\tThis is %s model trained on %s with test_size = %.2f' % (model_name, dataset_name, test_size))
    print('*' * 70 + '\n')
    model_manager = utils.ModelManager(dataset_name, test_size)
    try:
        trainset = model_manager.load_model('trainset')
        testset = model_manager.load_model('testset')
    except OSError:
        ratings = DataSet.load_dataset(name=dataset_name)
        trainset, testset = DataSet.train_test_split(ratings, test_size=test_size)
        model_manager.save_model(trainset, 'trainset')
        model_manager.save_model(testset, 'testset')
    '''Do you want to clean workspace and retrain model again?'''
    '''if you want to change test_size or retrain model, please set clean_workspace True'''
    model_manager.clean_workspace(clean)
    if model_name == 'UserCF':
        model = UserBasedCF()
    elif model_name == 'ItemCF':
        model = ItemBasedCF()
    elif model_name == 'Random':
        model = RandomPredict()
    elif model_name == 'MostPopular':
        model = MostPopular()
    elif model_name == 'UserCF-IIF':
        model = UserBasedCF(use_iif_similarity=True)
    elif model_name == 'ItemCF-IUF':
        model = ItemBasedCF(use_iuf_similarity=True)
    elif model_name == 'LFM':
        # K, epochs, alpha, lamb, n_rec_movie
        model = LFM(10, 20, 0.1, 0.01, 10)
    elif model_name == 'BasicSVD':
        # K, epochs, alpha, lamb, n_rec_movie
        model = BasicSVD(10, 20, 0.1, 0.01, 10)
    else:
        raise ValueError('No model named ' + model_name)
    model.fit(trainset)
    model.test(testset)
    rec_result = recommend_test(model, user_list)
    return rec_result



def recommend_test(model, user_list):
    rec_result = {}
    for user in user_list:
        recommend = model.recommend(str(user))
        print("recommend for userid = %s:" % user)
        print(recommend)
        rec_result[user] = recommend
    return rec_result
main_time = LogTime(words="Main Function")
dataset_name = 'ml-100k'

model_type = 'LFM'
test_size = 0.1
# 用户id
user_id_list = [5]
rec_result = run_model(model_type, dataset_name, test_size, False, user_id_list)

import json
file_path = "data/user_movie_feature/movies.csv"  # 请替换为实际的文件路径
dic_movie = {} # 用于存储解析后的数据
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        # 解析JSON对象
        record = json.loads(line.strip())
#         data.append(record)
#         print(record)
        dic_movie[str(record['id'])] = record['title']

rec_movie_name = [dic_movie[x] for x in rec_result[user_id_list[0]]]
print(rec_movie_name)

import pickle
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)
dic = {}
for name in rec_movie_name:
    if name in movie_data:
        print("find", name)
        dic[name] = movie_data[name]
        with open('show_movie_url_pic.pickle', 'wb') as f:
            pickle.dump(dic, f)
    else:
        print("not find", name)
with open('show_movie_url_pic.pickle', 'rb') as f:
        show_movie_data = pickle.load(f)
print(show_movie_data)

!python3 main2.py

**********************************************************************
	This is LFM model trained on ml-100k with test_size = 0.10
**********************************************************************

LFM start...

User origin similarity model has saved before.
Load model success...

Test recommendation system start...
 steps(0), 0.01 seconds have spent..
Test recommendation system success.
total  step number is 943
total 2.07 seconds have spent

precision=0.2038	recall=0.1915	coverage=0.3010	popularity=5.2464

recommend for userid = 5:
['82', '229', '449', '217', '210', '118', '588', '568', '550', '559']
['Jurassic Park (1993)', 'Star Trek III: The Search for Spock (1984)', 'Star Trek: The Motion Picture (1979)', "Bram Stoker's Dracula (1992)", 'Indiana Jones and the Last Crusade (1989)', 'Twister (1996)', 'Beauty and the Beast (1991)', 'Speed (1994)', 'Die Hard: With a Vengeance (1995)', 'Interview with the Vampire (1994)']
find Jurassic Park (1993)
find Star Trek III: The Search f

In [3]:
ls ./static/pictures

Amistad (1997).jpg
Batman (1989).jpg
Boogie Nights (1997).jpg
Chasing Amy (1997).jpg
Die Hard: With a Vengeance (1995).jpg
Emma (1996).jpg
Everyone Says I Love You (1996).jpg
Get Shorty (1995).jpg
Get Shorty.jpg
Highlander (1986).jpg
Jerry Maguire (1996).jpg
Mission: Impossible (1996).jpg
Mother (1996).jpg
Mrs. Brown (Her Majesty, Mrs. Brown) (1997).jpg
Seven Years in Tibet (1997).jpg
Speed (1994).jpg
Star Trek: First Contact (1996).jpg
Trainspotting (1996).jpg
True Lies (1994).jpg
Ulee's Gold (1997).jpg
titanic.jpg
venom.jpg


In [2]:
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)

In [3]:
movie_data

{"Ulee's Gold (1997)": ['https://www.imdb.com/title/tt0120402/?ref_=fn_al_tt_1',
  "./pictures/Ulee's Gold (1997).jpg"],
 'Chasing Amy (1997)': ['https://www.imdb.com/title/tt0118842/?ref_=fn_al_tt_1',
  './pictures/Chasing Amy (1997).jpg'],
 'Seven Years in Tibet (1997)': ['https://www.imdb.com/title/tt0120102/?ref_=fn_al_tt_1',
  './pictures/Seven Years in Tibet (1997).jpg'],
 'Amistad (1997)': ['https://www.imdb.com/title/tt0118607/?ref_=fn_al_tt_1',
  './pictures/Amistad (1997).jpg'],
 'Boogie Nights (1997)': ['https://www.imdb.com/title/tt0118749/?ref_=fn_al_tt_1',
  './pictures/Boogie Nights (1997).jpg'],
 'Everyone Says I Love You (1996)': ['https://www.imdb.com/title/tt0116242/?ref_=fn_al_tt_1',
  './pictures/Everyone Says I Love You (1996).jpg'],
 'Jerry Maguire (1996)': ['https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1',
  './pictures/Jerry Maguire (1996).jpg'],
 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': ['https://www.imdb.com/title/tt0119280/?ref_=fn_al_tt_1',
  '

In [ ]:
rec_movie_name

In [6]:
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)

In [7]:
movie_data

{"Ulee's Gold (1997)": ['https://www.imdb.com/title/tt0120402/?ref_=fn_al_tt_1',
  "./pictures/Ulee's Gold (1997).jpg"],
 'Chasing Amy (1997)': ['https://www.imdb.com/title/tt0118842/?ref_=fn_al_tt_1',
  './pictures/Chasing Amy (1997).jpg'],
 'Seven Years in Tibet (1997)': ['https://www.imdb.com/title/tt0120102/?ref_=fn_al_tt_1',
  './pictures/Seven Years in Tibet (1997).jpg'],
 'Amistad (1997)': ['https://www.imdb.com/title/tt0118607/?ref_=fn_al_tt_1',
  './pictures/Amistad (1997).jpg'],
 'Boogie Nights (1997)': ['https://www.imdb.com/title/tt0118749/?ref_=fn_al_tt_1',
  './pictures/Boogie Nights (1997).jpg'],
 'Everyone Says I Love You (1996)': ['https://www.imdb.com/title/tt0116242/?ref_=fn_al_tt_1',
  './pictures/Everyone Says I Love You (1996).jpg'],
 'Jerry Maguire (1996)': ['https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1',
  './pictures/Jerry Maguire (1996).jpg'],
 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': ['https://www.imdb.com/title/tt0119280/?ref_=fn_al_tt_1',
  '

In [9]:
with open('show_movie_url_pic.pickle', 'rb') as f:
        show_movie_data = pickle.load(f)

In [11]:
show_movie_data

{"Ulee's Gold (1997)": ['https://www.imdb.com/title/tt0120402/?ref_=fn_al_tt_1',
  "./pictures/Ulee's Gold (1997).jpg"],
 'Chasing Amy (1997)': ['https://www.imdb.com/title/tt0118842/?ref_=fn_al_tt_1',
  './pictures/Chasing Amy (1997).jpg'],
 'Seven Years in Tibet (1997)': ['https://www.imdb.com/title/tt0120102/?ref_=fn_al_tt_1',
  './pictures/Seven Years in Tibet (1997).jpg'],
 'Amistad (1997)': ['https://www.imdb.com/title/tt0118607/?ref_=fn_al_tt_1',
  './pictures/Amistad (1997).jpg'],
 'Boogie Nights (1997)': ['https://www.imdb.com/title/tt0118749/?ref_=fn_al_tt_1',
  './pictures/Boogie Nights (1997).jpg'],
 'Everyone Says I Love You (1996)': ['https://www.imdb.com/title/tt0116242/?ref_=fn_al_tt_1',
  './pictures/Everyone Says I Love You (1996).jpg'],
 'Jerry Maguire (1996)': ['https://www.imdb.com/title/tt0116695/?ref_=fn_al_tt_1',
  './pictures/Jerry Maguire (1996).jpg'],
 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)': ['https://www.imdb.com/title/tt0119280/?ref_=fn_al_tt_1',
  '